## Using the EWXPWSDB StationReadings Class

# <span style="color:red">clear all output before saving: db output contains passwords! </span>

this walks through process of

- creating a temporary DB
- using the StationReading class for existing station record to
 - get station info
 - mess around with timezones
 - pull readings by date interval
 - pull hourly and daily summaries of readings

In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Constants/Config

In [29]:

station_file = '../data/test_stations.tsv'
station_type = 'SPECTRUM'
station_code = 'EWXSPECTRUM01' 

In [30]:

from ewxpwsdb.db.database import create_temp_pg_engine, get_db_url, init_db, Session
from ewxpwsdb.db.models import WeatherStation, Reading, StationType, APIResponse


## create engine temp database

In [31]:

engine = create_temp_pg_engine(get_db_url(), name_prefix = 'notebook_testing')

temp_db_url = engine.url
print(temp_db_url.database)
init_db(engine,station_file)


notebook_testing_nxnynxjyxn
Station with type 'DAVIS' merged into the database
Station with type 'LOCOMOS' merged into the database
Station with type 'ONSET' merged into the database
Station with type 'RAINWISE' merged into the database
Station with type 'SPECTRUM' merged into the database
Station with type 'ZENTRA' merged into the database


In [ ]:
from sqlalchemy import Engine, text, inspect

with Session(engine) as session:
    results = session.exec(text('show timezone;')).all()
    print(results)
    
    

In [ ]:
from ewxpwsdb.station import Station

station = Station.from_station_code(station_code, engine)
print(station.weather_station.station_type)
print('does our station match the station type we expected?')
station.weather_station.station_type == station_type


## Get some data into our temp database

In [ ]:
from ewxpwsdb.collector import Collector

collector = Collector(station.weather_station, engine)
print(collector.station_code)

set up a time interval to pull data for

In [ ]:
from datetime import datetime, UTC, timedelta
from ewxpwsdb.time_intervals import UTCInterval

today_utc = datetime.now(UTC).date()

response_ids = collector.request_and_store_weather_data_utc(UTCInterval.one_day_interval())
n_readings = len(collector.current_reading_ids)
print(f"store {n_readings} readings for {station_code}")

somerex = collector.request_and_store_weather_data_utc(UTCInterval.one_day_interval(d = today_utc- timedelta(days = 1)))
n_readings = len(collector.current_reading_ids)
print(f"store {n_readings} readings for {station_code}")




In [ ]:
# 
response_ids = collector.catch_up()
n_readings = len(collector.current_reading_ids)
print(f"store {n_readings} readings for {station_code}")

## StationReadings class

In [ ]:
from ewxpwsdb.station import Station
from sqlmodel import select

with Session(engine) as session:            
    stmt = select(WeatherStation).where(WeatherStation.station_code == station_code)
    station_record:WeatherStation= session.exec(stmt).one()

station_record

In [ ]:
station_record.model_dump_json()

In [ ]:
test_station = Station.from_station_code(station_code, engine)
test_station.weather_station


In [ ]:
test_station.weather_station.id

In [ ]:
sql = test_station.weatherstation_plus_sql()
for line in sql.split('\n'):
    print(line)

In [ ]:
test_station.station_with_detail(engine).model_dump()

## Debugging WeatherStationDetail class

this is a class that is Pydantic Base model with class methods to create from station code and engine

In [ ]:
from ewxpwsdb.station import WeatherStationDetail
wsd:WeatherStationDetail = WeatherStationDetail.with_detail(station_code = station_code, engine = engine)
print(wsd.model_dump_json(indent = 4))
wsd.first_reading_datetime

In [ ]:
assert wsd.first_reading_datetime is not None

In [ ]:
# can we use supported vars to check things?  that field looks it's stored kinda wonky in data model, but it's JSON!

var_of_interest = 'lws'
import json
if var_of_interest in json.loads(wsd.supported_variables):
    print('var is in there -- we can do the thing!')

In [ ]:
from ewxpwsdb.station_readings import StationReadings


station_readings = StationReadings(station = test_station.weather_station, engine = engine)

print(station_readings.station.station_code)
print(station_readings.station.id)


In [ ]:
readings = station_readings.recent_readings()
print(readings)

create a date interval to use to pull data

In [ ]:

from zoneinfo import ZoneInfo
from datetime import date
from ewxpwsdb.time_intervals import DateInterval

dates = DateInterval(start = date(2024, 6, 8), end = date(2024, 6, 12), local_timezone = ZoneInfo("America/Detroit") )
utc_interval = dates.to_utc_datetime_interval()


Get more readings with collector, pull them with station_readings

In [ ]:
collector.request_and_store_weather_data_utc(utc_interval)
print(f"stored {len(collector.current_reading_ids)} readings ")



Testing complex SQL to pull hourly summary of readings by date

In [ ]:
from ewxpwsdb.db.summary_models import HourlySummary

start_date=dates.start
end_date=dates.end
sql_str = HourlySummary.sql_str(station_id= station.weather_station.id, local_start_date=dates.start, local_end_date=dates.end, station_timezone=station.weather_station.timezone)
for line in sql_str.split("\n"):
    print(line)

In [ ]:
one_day = DateInterval(start = date(2024, 6, 9), end = date(2024, 6, 10), local_timezone = ZoneInfo("America/Detroit") )

records = station_readings.hourly_summary(local_start_date = one_day.start, local_end_date=one_day.end )
print(len(records))
if (len(records) > 0 ):
    print(records[0])
    


### Clean up

remove test database

In [ ]:

from ewxpwsdb.db.database import drop_pg_db, list_pg_databases
from sqlalchemy.orm import close_all_sessions

# if collector:
#     collector._session.close()
#     collector._engine.dispose()

close_all_sessions()

print(f"attempting to drop db {engine.url.database}")
result = drop_pg_db(engine)
print(result)
engine.dispose()
list_pg_databases(host='localhost')

